# Restart & Modeling 
EfficientNetB0가 input shape이 (224,224,3)이라고 함

B7은 (600,600,3)

In [1]:
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB0 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Mac m1 pro tf with gpu
from tensorflow.python.compiler.mlcompute import mlcompute 
mlcompute.set_mlc_device(device_name='gpu') # Available options are 'cpu', 'gpu', and ‘any'.

# import custom datagenerator
from datagenerator import DataGenerator

import json

import pandas as pd
import numpy as np
from glob import glob

from PIL import Image
import cv2
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
data_path = './go_traffic_file'

train_path = data_path + '/train'
test_path = data_path + '/test'

new_image_directory = data_path + '/new_images'
new_train_image_directory = new_image_directory + '/train'
new_test_image_directory = new_image_directory + '/test'

action_information = pd.read_csv(data_path + '/action_information.csv')
sample_submission = pd.read_csv(data_path + '/sample_submission.csv')

classes = pd.get_dummies(action_information[['Label']], columns = ['Label']).to_numpy()

In [3]:
new_train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('/')[-1].split('_')[-1]))
new_train_image_directories[:5]

['./go_traffic_file/new_images/train/file_0',
 './go_traffic_file/new_images/train/file_1',
 './go_traffic_file/new_images/train/file_2',
 './go_traffic_file/new_images/train/file_3',
 './go_traffic_file/new_images/train/file_4']

In [4]:
train_answer = []
train_image_directories = sorted(glob(train_path + '/*'), key = lambda x :  int(x.split('_')[-1]))
for train_image_directory in train_image_directories:
    json_path = glob(train_image_directory + '/*.json')[0]
    js = json.load(open(json_path))
    action = js.get('action')
    train_answer.append(action)

In [5]:
list_IDs = []
labels = {}
stratify = []

for num, (new_train_image_directory, action) in tqdm(enumerate(zip(new_train_image_directories, train_answer)), total = len(new_train_image_directories)):
    image_paths = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
    image_len = len(image_paths)
    for image_path in image_paths:
        list_IDs.append(image_path)
        labels[image_path] = classes[action]
        stratify.append(action)
        
list_IDs = np.array(list_IDs)

100%|███████████████████████████████████████| 142/142 [00:00<00:00, 1370.02it/s]


In [6]:
train_idx, valid_idx = train_test_split(range(len(list_IDs)), test_size=0.2, stratify=stratify)

X_train = list_IDs[train_idx]
X_valid = list_IDs[valid_idx]

print(X_train.shape)
print(X_valid.shape)

(17774,)
(4444,)


In [7]:
training_generator = DataGenerator(X_train, labels, batch_size=64)
validation_generator = DataGenerator(X_valid, labels, batch_size=64)

In [8]:
optimizer = Adam(learning_rate=1e-2)

In [9]:
baseModel = EfficientNetB0(input_shape = (224,224,3), weights='imagenet', include_top=False,)
baseModel.trainable = False

model_in = Input(shape = (224,224,3))
base_model = baseModel(model_in)
head_model = MaxPooling2D(pool_size=(7,7))(base_model)
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(256, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(32, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(8, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)

model_out = Dense(classes.shape[1], activation='softmax')(head_model)

model = Model(inputs=model_in, outputs=model_out, name='EfficientNetB0_1')

model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

16711680/16705208 [==============================] - 8s 0us/step


In [11]:
model.summary()

Model: "EfficientNetB0_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 1280)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               327936    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)           

In [12]:
history = model.fit(
     training_generator, epochs=30,verbose=1, validation_data=validation_generator)

2021-12-01 15:16:53.874781: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-01 15:16:53.875039: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


Epoch 1/30
277/277 [==============================] - 296s 1s/step - loss: 3.5361 - accuracy: 0.1658 - val_loss: 3.4579 - val_accuracy: 0.1689
Epoch 2/30
277/277 [==============================] - 296s 1s/step - loss: 6.2696 - accuracy: 0.1750 - val_loss: 1.7919 - val_accuracy: 0.1671
Epoch 3/30
277/277 [==============================] - 295s 1s/step - loss: 1.7916 - accuracy: 0.1696 - val_loss: 1.7909 - val_accuracy: 0.1791
Epoch 4/30
277/277 [==============================] - 295s 1s/step - loss: 1.7917 - accuracy: 0.1753 - val_loss: 1.7906 - val_accuracy: 0.1787
Epoch 5/30
277/277 [==============================] - 294s 1s/step - loss: 1.7913 - accuracy: 0.1776 - val_loss: 1.7907 - val_accuracy: 0.1791
Epoch 6/30
277/277 [==============================] - 296s 1s/step - loss: 1.7907 - accuracy: 0.1736 - val_loss: 1.7906 - val_accuracy: 0.1784
Epoch 7/30
277/277 [==============================] - 297s 1s/step - loss: 1.7902 - accuracy: 0.1795 - val_loss: 1.7909 - val_accuracy: 0.1793

KeyboardInterrupt: 

In [21]:
training_generator = DataGenerator(X_train, labels, batch_size=128)
validation_generator = DataGenerator(X_valid, labels, batch_size=128)

In [22]:
baseModel = EfficientNetB0(input_shape = (224,224,3), weights='imagenet', include_top=False,)
baseModel.trainable = False

model_in = Input(shape = (224,224,3))
base_model = baseModel(model_in)
head_model = MaxPooling2D(pool_size=(7,7))(base_model)
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(256, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(32, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(16, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(8, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)

model_out = Dense(classes.shape[1], activation='softmax')(head_model)

model2 = Model(inputs=model_in, outputs=model_out, name='EfficientNetB0_2')

model2.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

In [23]:
model2.summary()

Model: "EfficientNetB0_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 1280)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               327936    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)           

In [24]:
history = model2.fit(
     training_generator, epochs=20,verbose=1, validation_data=validation_generator)

Epoch 1/20
 25/138 [====>.........................] - ETA: 38:49 - loss: 16.6163 - accuracy: 0.1700

KeyboardInterrupt: 

In [ ]:
batch_size = 32
learning_rate = 1e-3
optimizer = Adam(learning_rate=learning_rate)
model_name ='EfficientNetB0_3'
epochs = 50

training_generator = DataGenerator(X_train, labels, batch_size=batch_size)
validation_generator = DataGenerator(X_valid, labels, batch_size=batch_size)

baseModel = EfficientNetB0(input_shape = (224,224,3), weights='imagenet', include_top=False,)
baseModel.trainable = False

model_in = Input(shape = (224,224,3))
base_model = baseModel(model_in)
head_model = MaxPooling2D(pool_size=(7,7))(base_model)
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(256, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(32, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(8, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)

model_out = Dense(classes.shape[1], activation='softmax')(head_model)

model = Model(inputs=model_in, outputs=model_out, name=model_name)

model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

model.summary()

history = model.fit(
     training_generator, epochs=epochs,verbose=1, validation_data=validation_generator)

Model: "EfficientNetB0_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1, 1, 1280)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               327936    
_________________________________________________________________
dropout_13 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)           